# **Automatation of the CLI commands of the Snakemake workflow**

* Environment to use: [Greference_tools](code/environments/Greference_tools.yml)

In [1]:
## Libreries
import subprocess
import os
import re

In [2]:
## lists for the loops
chr_list        = ["chr3", "chr5", "chr7", "chr12", "chr17"]

sample_list     = ["ERR696683", "ERR753368", "ERR753369" ,"ERR753370" ,"ERR753371" ,"ERR753372", 
                  "ERR753373", "ERR753374", "ERR753375", "ERR753376", "ERR753377", "ERR753378"]


1. **rule download_data**

In [7]:
subprocess.run(["snakemake", "--cores", "1", "download_data"])

Building DAG of jobs...
Nothing to be done (all requested files are present and up to date).
Complete log: .snakemake/log/2023-08-28T131200.585023.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'download_data'], returncode=0)

* rule pre_processing

Filtering the chromosomes, make sure that the chromosomes of the config file are **ch3**. 

This rule is preatty long too... good luck

When it finished, it is recomended to delete the SAM files not oredered.

In [8]:
## Running the 3º chromosome
subprocess.run(["snakemake", "--cores", "1", "data/original_bam/filtering/ERR696683_chr3_sorted.bam"])

## Running the rest of the chromosomes
for chr in range(len(chr_list) - 1):
    chr1 = chr_list[chr]
    chr2 = chr_list[chr + 1]
    
    print("""
          Chromosomes transformed:
          """, chr1, chr2)
    
    subprocess.run(["sed", "-i", "6,+12s/" + chr1 + "/" + chr2 + "/g", "config.yaml"])
    subprocess.run(["sed", "-i", "39s/" + chr1 + "/" + chr2 + "/g", "config.yaml"])

    subprocess.run(["snakemake", "--cores", "1", "data/original_bam/filtering/ERR696683_" + chr2 + "_sorted.bam"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Aug 28 13:12:25 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr3_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr3_sorted.bam
    wildcards: sample=ERR696683_chr3
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 84123 singletons
[M::bam2fq_mainloop] processed 4225255 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] disca


          Chromosomes transformed:
           chr3 chr5


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Aug 28 13:55:10 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr5_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr5_sorted.bam
    wildcards: sample=ERR696683_chr5
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 74911 singletons
[M::bam2fq_mainloop] processed 3565319 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] disca


          Chromosomes transformed:
           chr5 chr7


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Aug 28 14:33:57 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr7_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr7_sorted.bam
    wildcards: sample=ERR696683_chr7
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 73902 singletons
[M::bam2fq_mainloop] processed 3944452 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] disca


          Chromosomes transformed:
           chr7 chr12


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Aug 28 15:13:49 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr12_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr12_sorted.bam
    wildcards: sample=ERR696683_chr12
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 64449 singletons
[M::bam2fq_mainloop] processed 3693551 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] di


          Chromosomes transformed:
           chr12 chr17


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Aug 28 15:53:04 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr17_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr17_sorted.bam
    wildcards: sample=ERR696683_chr17
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 49649 singletons
[M::bam2fq_mainloop] processed 3373755 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] di

In [3]:
## reset config 
subprocess.run(["sed", "-i", "6,+12s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i", "39s/chr17/chr3/g", "config.yaml"])

CompletedProcess(args=['sed', '-i', '39s/chr17/chr3/g', 'config.yaml'], returncode=0)

* rules reference_genome; fastqc; fastp; fastqc_trimmed; bwa_mapping; sam_to_bam; delete_duplicates; extracting_variants; vep_install; vep_cli; parsing_dataR

In [10]:
## Running the workflow for the 3º chormosome

subprocess.run(["snakemake", "--cores", "1", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr3_1_fastqc.html",   
                    "data/processed/" + sample + "_chr3_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr3_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr3_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr3_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr3_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr3.vcf"])   

## Downloading VEP
subprocess.run(["snakemake", "--cores", "1", "--use-conda", "vep_install_db"])
## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr3.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Mon Aug 28 16:47:08 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Missing output files: data/reference/genome.fa
    resources: tmpdir=/tmp

rm: no se puede borrar 'data/reference/genome.fa*': No existe el archivo o el directorio
--2023-08-28 16:47:08--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.3.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 

==> If you hadn't a previous reference in the directory and there is an ERROR, is NORMAL <==



193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 59761837 (57M) [application/x-gzip]
Guardando como: ‘data/reference/genome.fa.gz’

     0K .......... .......... .......... .......... ..........  0%  452K 2m9s
    50K .......... .......... .......... .......... ..........  0% 1,26M 87s
   100K .......... .......... .......... .......... ..........  0% 2,53M 65s
   150K .......... .......... .......... .......... ..........  0% 1,30M 60s
   200K .......... .......... .......... .......... ..........  0% 2,73M 52s
   250K .......... .......... .......... .......... ..........  0% 17,0M 44s
   300K .......... .......... .......... .......... ..........  0% 65,0M 38s
   350K .......... .......... .......... .......... ..........  0% 80,2M 33s
   400K .......... .......... .......... .......... ..........  0% 1,37M 34s
   450K .......... .......... .......... .......... ..........

application/gzip


Started analysis of ERR696683_chr3_1.fastq.gz
Approx 5% complete for ERR696683_chr3_1.fastq.gz
Approx 10% complete for ERR696683_chr3_1.fastq.gz
Approx 15% complete for ERR696683_chr3_1.fastq.gz
Approx 20% complete for ERR696683_chr3_1.fastq.gz
Approx 25% complete for ERR696683_chr3_1.fastq.gz
Approx 30% complete for ERR696683_chr3_1.fastq.gz
Approx 35% complete for ERR696683_chr3_1.fastq.gz
Approx 40% complete for ERR696683_chr3_1.fastq.gz
Approx 45% complete for ERR696683_chr3_1.fastq.gz
Approx 50% complete for ERR696683_chr3_1.fastq.gz
Approx 55% complete for ERR696683_chr3_1.fastq.gz
Approx 60% complete for ERR696683_chr3_1.fastq.gz
Approx 65% complete for ERR696683_chr3_1.fastq.gz
Approx 70% complete for ERR696683_chr3_1.fastq.gz
Approx 75% complete for ERR696683_chr3_1.fastq.gz
Approx 80% complete for ERR696683_chr3_1.fastq.gz
Approx 85% complete for ERR696683_chr3_1.fastq.gz
Approx 90% complete for ERR696683_chr3_1.fastq.gz
Approx 95% complete for ERR696683_chr3_1.fastq.gz


...


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 25 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 3_179203851_C/A       3:1792038… A      ENSG… ENST00… Transcript   intron_var…
2 3_179203851_C/A       3:1792038… A      ENSG… ENST00… Transcript   downstream…
3 3_179203851_C/A       3:1792038… A      ENSG… ENST00… Transcript   downstream…
4 3_179203851_C/A       3:1792038… A      ENSG… ENST00… Transcript   intron_var…
5 3_179203851_C/A       3:1792038… A      ENSG… ENST00… Transcript   intron_var…
6 3_179203851_C/A       3:1792038… A      ENSG… ENST00… Transcript   intron_var…
# ℹ 19 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Mon Aug 28 18:32:42 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-28T183238.963328.snakemake.log


# A tibble: 86 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   intron_var…
2 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   downstream…
3 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   downstream…
4 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   intron_var…
5 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   upstream_g…
6 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   intron_var…
# ℹ 80 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele

CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

## Changing the config file to do the chromosome 5

In [4]:
subprocess.run(["sed", "-i", "6,+12s/chr3/chr5/g", "config.yaml"])
subprocess.run(["sed", "-i", "24,+12s/chr3/chr5/g", "config.yaml"])
subprocess.run(["sed", "-i","39s/chr3/chr5/g", "config.yaml"])
subprocess.run(["sed", "-i", "46s/chromosome.3.fa.gz/chromosome.5.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "65s/PIK3CA/APC/g", "config.yaml"])


CompletedProcess(args=['sed', '-i', '65s/PIK3CA/APC/g', 'config.yaml'], returncode=0)

In [5]:
## Running the workflow for the 5º chormosome

subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr5_1_fastqc.html",   
                    "data/processed/" + sample + "_chr5_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr5_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr5_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr5_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr5_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr5.vcf"])   

## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr5.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Mon Aug 28 20:40:40 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Forced execution
    resources: tmpdir=/tmp

--2023-08-28 20:40:40--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.5.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 54485176 (52M) [application/x-gzip]
Guardando como: ‘data

application/gzip


Started analysis of ERR696683_chr5_1_fastp.fastq.gz
Approx 5% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 10% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 15% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 20% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 25% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 30% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 35% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 40% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 45% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 50% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 55% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 60% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 65% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 70% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 75% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 80% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 85% complete for ERR696683_chr5_1_fastp.fastq.

Analysis complete for ERR696683_chr5_1_fastp.fastq.gz
application/gzip


Started analysis of ERR696683_chr5_2_fastp.fastq.gz
Approx 5% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 10% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 15% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 20% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 25% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 30% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 35% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 40% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 45% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 50% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 55% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 60% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 65% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 70% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 75% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 80% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 85% complete for ERR696683_chr5_2_fastp.fastq.

...

# A tibble: 59 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 5_112729129_G/C       5:1127291… C      ENSG… ENST00… Transcript   intron_var…
2 5_112729129_G/C       5:1127291… C      ENSG… ENST00… Transcript   intron_var…
3 5_112729129_G/C       5:1127291… C      ENSG… ENST00… Transcript   intron_var…
4 5_112775373_G/A       5:1127753… A      ENSG… ENST00… Transcript   intron_var…
5 5_112775373_G/A       5:1127753… A      ENSG… ENST00… Transcript   intron_var…
6 5_112775373_G/A       5:1127753… A      ENSG… ENST00… Transcript   intron_var…
# ℹ 53 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 71 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
2 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
3 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
4 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
5 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
6 5_112778038_C/T       5:1127780… T      ENSG… ENST00… Transcript   intron_var…
# ℹ 65 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Mon Aug 28 22:11:22 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-28T221119.790527.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

## Changing the config file to do the chromosome 7 

In [6]:
subprocess.run(["sed", "-i", "6,+12s/chr5/chr7/g", "config.yaml"])
subprocess.run(["sed", "-i", "24,+12s/chr5/chr7/g", "config.yaml"])
subprocess.run(["sed", "-i","39s/chr5/chr7/g", "config.yaml"])
subprocess.run(["sed", "-i", "46s/chromosome.5.fa.gz/chromosome.7.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "65s/APC/BRAF/g", "config.yaml"])

CompletedProcess(args=['sed', '-i', '65s/APC/BRAF/g', 'config.yaml'], returncode=0)

In [7]:
## Running the workflow for the 7º chormosome

subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr7_1_fastqc.html",   
                    "data/processed/" + sample + "_chr7_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr7_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr7_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr7_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr7_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr7.vcf"])   

## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr7.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Mon Aug 28 22:55:34 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Forced execution
    resources: tmpdir=/tmp

--2023-08-28 22:55:34--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.7.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 47416846 (45M) [application/x-gzip]
Guardando como: ‘data

application/gzip


Started analysis of ERR696683_chr7_1.fastq.gz
Approx 5% complete for ERR696683_chr7_1.fastq.gz
Approx 10% complete for ERR696683_chr7_1.fastq.gz
Approx 15% complete for ERR696683_chr7_1.fastq.gz
Approx 20% complete for ERR696683_chr7_1.fastq.gz
Approx 25% complete for ERR696683_chr7_1.fastq.gz
Approx 30% complete for ERR696683_chr7_1.fastq.gz
Approx 35% complete for ERR696683_chr7_1.fastq.gz
Approx 40% complete for ERR696683_chr7_1.fastq.gz
Approx 45% complete for ERR696683_chr7_1.fastq.gz
Approx 50% complete for ERR696683_chr7_1.fastq.gz
Approx 55% complete for ERR696683_chr7_1.fastq.gz
Approx 60% complete for ERR696683_chr7_1.fastq.gz
Approx 65% complete for ERR696683_chr7_1.fastq.gz
Approx 70% complete for ERR696683_chr7_1.fastq.gz
Approx 75% complete for ERR696683_chr7_1.fastq.gz
Approx 80% complete for ERR696683_chr7_1.fastq.gz
Approx 85% complete for ERR696683_chr7_1.fastq.gz
Approx 90% complete for ERR696683_chr7_1.fastq.gz
Approx 95% complete for ERR696683_chr7_1.fastq.gz


Analysis complete for ERR696683_chr7_1.fastq.gz
application/gzip


Started analysis of ERR696683_chr7_2.fastq.gz
Approx 5% complete for ERR696683_chr7_2.fastq.gz
Approx 10% complete for ERR696683_chr7_2.fastq.gz
Approx 15% complete for ERR696683_chr7_2.fastq.gz
Approx 20% complete for ERR696683_chr7_2.fastq.gz
Approx 25% complete for ERR696683_chr7_2.fastq.gz
Approx 30% complete for ERR696683_chr7_2.fastq.gz
Approx 35% complete for ERR696683_chr7_2.fastq.gz
Approx 40% complete for ERR696683_chr7_2.fastq.gz
Approx 45% complete for ERR696683_chr7_2.fastq.gz
Approx 50% complete for ERR696683_chr7_2.fastq.gz
Approx 55% complete for ERR696683_chr7_2.fastq.gz
Approx 60% complete for ERR696683_chr7_2.fastq.gz
Approx 65% complete for ERR696683_chr7_2.fastq.gz
Approx 70% complete for ERR696683_chr7_2.fastq.gz
Approx 75% complete for ERR696683_chr7_2.fastq.gz
Approx 80% complete for ERR696683_chr7_2.fastq.gz
Approx 85% complete for ERR696683_chr7_2.fastq.gz
Approx 90% complete for ERR696683_chr7_2.fastq.gz
Approx 95% complete for ERR696683_chr7_2.fastq.gz


...


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 0 × 44
# ℹ 44 variables: #uploaded_variation <chr>, location <chr>, allele <chr>,
#   gene <chr>, feature <chr>, feature_type <chr>, consequence <chr>,
#   cdna_position <chr>, cds_position <chr>, protein_position <chr>,
#   amino_acids <chr>, codons <chr>, existing_variation <chr>, impact <chr>,
#   distance <chr>, strand <chr>, flags <chr>, variant_class <chr>,
#   symbol <chr>, symbol_source <chr>, hgnc_id <chr>, biotype <chr>,
#   mane_select <chr>, mane_plus_clinical <chr>, tsl <chr>, appris <chr>, …



── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Tue Aug 29 00:27:19 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-29T002715.519399.snakemake.log


# A tibble: 26 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   synonymous…
2 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   synonymous…
3 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   synonymous…
4 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   3_prime_UT…
5 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   3_prime_UT…
6 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   non_coding…
# ℹ 20 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele

CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

## Changing the config file to do the chromosome 12 

In [4]:
subprocess.run(["sed", "-i", "6,+12s/chr7/chr12/g", "config.yaml"])
subprocess.run(["sed", "-i", "24,+12s/chr7/chr12/g", "config.yaml"])
subprocess.run(["sed", "-i","39s/chr7/chr12/g", "config.yaml"])
subprocess.run(["sed", "-i", "46s/chromosome.7.fa.gz/chromosome.12.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "65s/BRAF/KRAS/g", "config.yaml"])

CompletedProcess(args=['sed', '-i', '65s/BRAF/KRAS/g', 'config.yaml'], returncode=0)

In [5]:
## Running the workflow for the 12º chormosome

subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr12_1_fastqc.html",   
                    "data/processed/" + sample + "_chr12_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr12_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr12_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr12_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr12_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr12.vcf"])   

## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr12.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Tue Aug 29 10:24:41 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Forced execution
    resources: tmpdir=/tmp

--2023-08-29 10:24:41--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 39795542 (38M) [application/x-gzip]
Guardando como: ‘dat

application/gzip


Started analysis of ERR696683_chr12_1_fastp.fastq.gz
Approx 5% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 10% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 15% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 20% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 25% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 30% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 35% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 40% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 45% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 50% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 55% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 60% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 65% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 70% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 75% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 80% complete for ERR696683_chr12_1_fastp.fastq.gz
Approx 85% complete for ERR696683_ch

Analysis complete for ERR696683_chr12_1_fastp.fastq.gz
application/gzip


Started analysis of ERR696683_chr12_2_fastp.fastq.gz
Approx 5% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 10% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 15% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 20% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 25% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 30% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 35% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 40% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 45% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 50% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 55% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 60% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 65% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 70% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 75% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 80% complete for ERR696683_chr12_2_fastp.fastq.gz
Approx 85% complete for ERR696683_ch

...

# A tibble: 24 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
2 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
3 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
4 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
5 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
6 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
# ℹ 18 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 12 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
2 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
3 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
4 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
5 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
6 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
# ℹ 6 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_selec


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Tue Aug 29 11:44:33 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-29T114430.423390.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

## Changing the config file to do the chromosome 17 

In [4]:
subprocess.run(["sed", "-i", "6,+12s/chr12/chr17/g", "config.yaml"])
subprocess.run(["sed", "-i", "24,+12s/chr12/chr17/g", "config.yaml"])
subprocess.run(["sed", "-i","39s/chr12/chr17/g", "config.yaml"])
subprocess.run(["sed", "-i", "46s/chromosome.12.fa.gz/chromosome.17.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "65s/KRAS/TP53/g", "config.yaml"])


CompletedProcess(args=['sed', '-i', '65s/KRAS/TP53/g', 'config.yaml'], returncode=0)

In [5]:
## Running the workflow for the 17º chormosome

subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr17_1_fastqc.html",   
                    "data/processed/" + sample + "_chr17_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr17_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr17_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr17_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr17_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr17.vcf"])   

## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr17.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Tue Aug 29 17:50:29 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Forced execution
    resources: tmpdir=/tmp

--2023-08-29 17:50:29--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.17.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 24104573 (23M) [application/x-gzip]
Guardando como: ‘dat

application/gzip


Started analysis of ERR696683_chr17_1.fastq.gz
Approx 5% complete for ERR696683_chr17_1.fastq.gz
Approx 10% complete for ERR696683_chr17_1.fastq.gz
Approx 15% complete for ERR696683_chr17_1.fastq.gz
Approx 20% complete for ERR696683_chr17_1.fastq.gz
Approx 25% complete for ERR696683_chr17_1.fastq.gz
Approx 30% complete for ERR696683_chr17_1.fastq.gz
Approx 35% complete for ERR696683_chr17_1.fastq.gz
Approx 40% complete for ERR696683_chr17_1.fastq.gz
Approx 45% complete for ERR696683_chr17_1.fastq.gz
Approx 50% complete for ERR696683_chr17_1.fastq.gz
Approx 55% complete for ERR696683_chr17_1.fastq.gz
Approx 60% complete for ERR696683_chr17_1.fastq.gz
Approx 65% complete for ERR696683_chr17_1.fastq.gz
Approx 70% complete for ERR696683_chr17_1.fastq.gz
Approx 75% complete for ERR696683_chr17_1.fastq.gz
Approx 80% complete for ERR696683_chr17_1.fastq.gz
Approx 85% complete for ERR696683_chr17_1.fastq.gz
Approx 90% complete for ERR696683_chr17_1.fastq.gz
Approx 95% complete for ERR696683_ch

Analysis complete for ERR696683_chr17_1.fastq.gz
application/gzip


Started analysis of ERR696683_chr17_2.fastq.gz
Approx 5% complete for ERR696683_chr17_2.fastq.gz
Approx 10% complete for ERR696683_chr17_2.fastq.gz
Approx 15% complete for ERR696683_chr17_2.fastq.gz
Approx 20% complete for ERR696683_chr17_2.fastq.gz
Approx 25% complete for ERR696683_chr17_2.fastq.gz
Approx 30% complete for ERR696683_chr17_2.fastq.gz
Approx 35% complete for ERR696683_chr17_2.fastq.gz
Approx 40% complete for ERR696683_chr17_2.fastq.gz
Approx 45% complete for ERR696683_chr17_2.fastq.gz
Approx 50% complete for ERR696683_chr17_2.fastq.gz
Approx 55% complete for ERR696683_chr17_2.fastq.gz
Approx 60% complete for ERR696683_chr17_2.fastq.gz
Approx 65% complete for ERR696683_chr17_2.fastq.gz
Approx 70% complete for ERR696683_chr17_2.fastq.gz
Approx 75% complete for ERR696683_chr17_2.fastq.gz
Approx 80% complete for ERR696683_chr17_2.fastq.gz
Approx 85% complete for ERR696683_chr17_2.fastq.gz
Approx 90% complete for ERR696683_chr17_2.fastq.gz
Approx 95% complete for ERR696683_ch

...


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 52 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
2 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
3 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
4 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
5 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
6 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
# ℹ 46 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Tue Aug 29 18:52:39 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-29T185235.823072.snakemake.log


# A tibble: 149 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   downstream…
2 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   intron_var…
3 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   intron_var…
4 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   downstream…
5 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   downstream…
6 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   downstream…
# ℹ 143 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_se

CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

In [6]:
## reset config 
subprocess.run(["sed", "-i", "6,+12s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i", "24,+12s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i","39s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i", "46s/chromosome.17.fa.gz/chromosome.3.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "65s/TP53/PIK3CA/g", "config.yaml"])

CompletedProcess(args=['sed', '-i', '65s/TP53/PIK3CA/g', 'config.yaml'], returncode=0)

## **Last rule**

### **rule R_plotting**

Plotting the data of the 5 gene tables

In [7]:
subprocess.run(["snakemake", "--cores", "1", "R_plotting"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job           count    min threads    max threads
----------  -------  -------------  -------------
R_plotting        1              1              1
total             1              1              1

Select jobs to execute...

[Tue Aug 29 18:54:28 2023]
rule R_plotting:
    input: code/06final_plot.R
    output: results/biostatistics/plots/final_plot.png, results/biostatistics/plots/other_plot.png
    jobid: 0
    reason: Missing output files: results/biostatistics/plots/final_plot.png, results/biostatistics/plots/other_plot.png
    resources: tmpdir=/tmp



[1] "##==========================##"
[1] "##===> WORK FINISHED!!! <===##"
[1] "##==========================##"
THE SCRIPT RAN WELL congrats :)


[Tue Aug 29 18:54:31 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-29T185428.517384.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'R_plotting'], returncode=0)